In [3]:
from helper import get_training_data, train_lasso_regression, plot_result_correlation, get_best_lasso_model, train_xgboost,  get_best_pca_lasso_model, train_mean_model
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn import metrics
from sklearn.preprocessing import RobustScaler
import matplotlib.pyplot as plt
from helper import prepare_socios
from scipy import stats

%matplotlib inline
%load_ext autoreload
%autoreload 2
pd.options.mode.chained_assignment = None
import warnings
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Best models

#### Unemployment rate

In [9]:
results = pd.DataFrame(columns = ['improvement_lasso', 'improvement_lasso_boosted', 'improvement_pca_lasso', 'improvement_pca_lasso_boosted', 'improvement_mean_model'])
results_mse = pd.DataFrame(columns = ['mse_naive','mse_lasso', 'mse_lasso_boosted', 'mse_pca_lasso', 'mse_pca_lasso_boosted', 'mse_mean_model'])
results_r2 = pd.DataFrame(columns = ['r2_naive','r2_lasso', 'r2_lasso_boosted', 'r2_pca_lasso', 'r2_pca_lasso_boosted', 'r2_mean_model'])
p_list= []
target = 'unemployment_rate'
dens_type = 'count'
radius = 1000

for city in ['marseille', 'lyon', 'paris']:
    country = 'FR'
    agg = get_training_data(city, country, radius, dens_type,  2015)
    
    # Lasso:
    predicts = get_best_lasso_model(agg=agg, target=target, city=city, country=country, socio_year=2015, density_type=dens_type, radius = radius, output ='predicts')
    predicts_master = predicts.copy()
    predicts_master = predicts_master.rename(columns = {'y_pred': 'y_pred_lasso'})

    # Lasso Boosted
    cols = get_best_lasso_model(agg=agg, target=target, city=city, country=country, socio_year=2015, density_type=dens_type, radius = radius, output ='used_columns')
    predicts = train_xgboost(agg, target, cols, 'predicts')
    predicts_master.loc[:, 'y_pred_lasso_boosted'] = predicts.y_pred

    # PCA Lasso
    predicts = get_best_pca_lasso_model(agg=agg, target=target, city=city, country=country, socio_year=2015, density_type=dens_type, radius = radius,output = 'predicts')
    predicts_master.loc[:,'y_pred_pca_lasso'] = predicts.y_pred

    # PCA Lasso boosted
    comps = get_best_pca_lasso_model(agg=agg,target=target, city=city, country=country, socio_year=2015,  density_type=dens_type, radius = radius, output = 'components')
    cols = get_best_pca_lasso_model(agg=agg,target=target, city=city, country=country, socio_year=2015,  density_type=dens_type, radius = radius, output = 'used_columns')
    reduced_data = pd.DataFrame(comps)
    reduced_data = reduced_data.join(agg.iloc[:,-5:])
    predicts = train_xgboost(reduced_data, target, cols, 'predicts')
    predicts_master.loc[:,'y_pred_pca_lasso_boosted'] = predicts.y_pred

    #predicts_master.loc[:, 'y_pred_mean_model'] = predicts_master[['y_pred_lasso', 'y_pred_lasso_boosted', 'y_pred_pca_lasso', 'y_pred_pca_lasso_boosted']].mean(axis = 1)
    predicts_master.loc[:, 'y_pred_mean_model'] = predicts_master[['y_pred_lasso_boosted', 'y_pred_pca_lasso_boosted']].mean(axis = 1)


    targets = ['unemployment_rate', 'income_levels', 'foreign_nationals']
    scaler = get_training_data(city, country, radius, dens_type, 2015, 'scaler')
    scaler_new = RobustScaler()
    scaler_new.center_, scaler_new.scale_ = scaler.center_[targets.index(target)], scaler.scale_[targets.index(target)]
    predicts_master = pd.DataFrame(scaler_new.inverse_transform(predicts_master), columns =predicts_master.columns)


    naive_mse = metrics.mean_squared_error(predicts_master.y_test, predicts_master.naive) 
    mse_lasso = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_lasso)
    mse_lasso_boosted = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_lasso_boosted)
    mse_pca_lasso = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_pca_lasso)
    mse_pca_lasso_boosted = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_pca_lasso_boosted)
    mse_mean_model = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_mean_model)
    
    naive_r2 = metrics.r2_score(predicts_master.y_test, predicts_master.naive) 
    r2_lasso = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_lasso)
    r2_lasso_boosted = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_lasso_boosted)
    r2_pca_lasso = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_pca_lasso)
    r2_pca_lasso_boosted = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_pca_lasso_boosted)
    r2_mean_model = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_mean_model)

    improvement_lasso = 100 - (mse_lasso/naive_mse)*100
    improvement_lasso_boosted = 100 - (mse_lasso_boosted /naive_mse)*100
    improvement_pca_lasso = 100 - (mse_pca_lasso/naive_mse)*100
    improvement_pca_lasso_boosted = 100 - (mse_pca_lasso_boosted/naive_mse)*100
    improvement_mean_model = 100 - (mse_mean_model/naive_mse)*100
    results.loc[city,:]=[improvement_lasso, improvement_lasso_boosted, improvement_pca_lasso, improvement_pca_lasso_boosted, improvement_mean_model]
    results_mse.loc[city,:]=[naive_mse, mse_lasso, mse_lasso_boosted, mse_pca_lasso, mse_pca_lasso_boosted, mse_mean_model]
    results_r2.loc[city,:]=[naive_r2, r2_lasso, r2_lasso_boosted, r2_pca_lasso, r2_pca_lasso_boosted, r2_mean_model]
    
    residuals = pd.DataFrame(columns = ['residuals_naive', 'residuals_m5'])
    residuals.loc[:,'residuals_naive'] = (predicts_master.y_test-predicts_master.naive)**2
    residuals.loc[:,'residuals_m5'] = (predicts_master.y_test-predicts_master.y_pred_mean_model)**2

    t_stat, p = stats.ttest_ind(residuals['residuals_naive'], residuals['residuals_m5'])
    print(f'TTest for residuals of model in {city}: t={t_stat}, p={p}')
    p_list.append(p)
    
for city in ['hamburg', 'berlin', 'bremen']:
    country = 'DE'
    agg = get_training_data(city, country, radius, dens_type,  2015)
    
    # Lasso:
    predicts = get_best_lasso_model(agg=agg, target=target, city=city, country=country, socio_year=2015, density_type=dens_type, radius = radius, output ='predicts')
    predicts_master = predicts.copy()
    predicts_master = predicts_master.rename(columns = {'y_pred': 'y_pred_lasso'})

    # Lasso Boosted
    cols = get_best_lasso_model(agg=agg, target=target, city=city, country=country, socio_year=2015, density_type=dens_type, radius = radius, output ='used_columns')
    predicts = train_xgboost(agg, target, cols, 'predicts')
    predicts_master.loc[:, 'y_pred_lasso_boosted'] = predicts.y_pred

    # PCA Lasso
    predicts = get_best_pca_lasso_model(agg=agg, target=target, city=city, country=country, socio_year=2015,  density_type=dens_type, radius = radius,output = 'predicts')
    predicts_master.loc[:,'y_pred_pca_lasso'] = predicts.y_pred

    # PCA Lasso boosted
    comps = get_best_pca_lasso_model(agg=agg,target=target, city=city, country=country, socio_year=2015,  density_type=dens_type, radius = radius, output = 'components')
    cols = get_best_pca_lasso_model(agg=agg,target=target, city=city, country=country, socio_year=2015, density_type=dens_type, radius = radius, output = 'used_columns')
    reduced_data = pd.DataFrame(comps)
    reduced_data = reduced_data.join(agg.iloc[:,-5:])
    predicts = train_xgboost(reduced_data, target, cols, 'predicts')
    predicts_master.loc[:,'y_pred_pca_lasso_boosted'] = predicts.y_pred

    #predicts_master.loc[:, 'y_pred_mean_model'] = predicts_master[['y_pred_lasso', 'y_pred_lasso_boosted', 'y_pred_pca_lasso', 'y_pred_pca_lasso_boosted']].mean(axis = 1)
    predicts_master.loc[:, 'y_pred_mean_model'] = predicts_master[['y_pred_lasso_boosted', 'y_pred_pca_lasso_boosted']].mean(axis = 1)

    targets = ['unemployment_rate', 'income_levels', 'foreign_nationals']
    scaler = get_training_data(city, country, radius, dens_type, 2015, 'scaler')
    scaler_new = RobustScaler()
    scaler_new.center_, scaler_new.scale_ = scaler.center_[targets.index(target)], scaler.scale_[targets.index(target)]
    predicts_master = pd.DataFrame(scaler_new.inverse_transform(predicts_master), columns =predicts_master.columns)


    naive_mse = metrics.mean_squared_error(predicts_master.y_test, predicts_master.naive) 
    mse_lasso = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_lasso)
    mse_lasso_boosted = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_lasso_boosted)
    mse_pca_lasso = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_pca_lasso)
    mse_pca_lasso_boosted = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_pca_lasso_boosted)
    mse_mean_model = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_mean_model)
    
    naive_r2 = metrics.r2_score(predicts_master.y_test, predicts_master.naive) 
    r2_lasso = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_lasso)
    r2_lasso_boosted = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_lasso_boosted)
    r2_pca_lasso = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_pca_lasso)
    r2_pca_lasso_boosted = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_pca_lasso_boosted)
    r2_mean_model = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_mean_model)

    improvement_lasso = 100 - (mse_lasso/naive_mse)*100
    improvement_lasso_boosted = 100 - (mse_lasso_boosted /naive_mse)*100
    improvement_pca_lasso = 100 - (mse_pca_lasso/naive_mse)*100
    improvement_pca_lasso_boosted = 100 - (mse_pca_lasso_boosted/naive_mse)*100
    improvement_mean_model = 100 - (mse_mean_model/naive_mse)*100
    results.loc[city,:]=[improvement_lasso, improvement_lasso_boosted, improvement_pca_lasso, improvement_pca_lasso_boosted, improvement_mean_model]
    results_mse.loc[city,:]=[naive_mse, mse_lasso, mse_lasso_boosted, mse_pca_lasso, mse_pca_lasso_boosted, mse_mean_model]
    results_r2.loc[city,:]=[naive_r2, r2_lasso, r2_lasso_boosted, r2_pca_lasso, r2_pca_lasso_boosted, r2_mean_model]
    
    residuals = pd.DataFrame(columns = ['residuals_naive', 'residuals_m5'])
    residuals.loc[:,'residuals_naive'] = (predicts_master.y_test-predicts_master.naive)**2
    residuals.loc[:,'residuals_m5'] = (predicts_master.y_test-predicts_master.y_pred_mean_model)**2

    t_stat, p = stats.ttest_ind(residuals['residuals_naive'], residuals['residuals_m5'])
    print(f'TTest for residuals of model in {city}: t={t_stat}, p={p}')
    #results_r2.loc[city,'p_values'] = p
    p_list.append(p)
    
#results.loc['mean'] = results.mean()
results_r2.loc[:,'p_values'] = p_list
results

shape of training data (344, 243)
0.1
0.1
shape before pca: (344, 236)
number of pca components: 69
shape after pca: (344, 69)
0.1
shape before pca: (344, 236)
number of pca components: 69
shape after pca: (344, 69)
0.1
shape before pca: (344, 236)
number of pca components: 69
shape after pca: (344, 69)
0.1
shape of training data (344, 243)
TTest for residuals of model in marseille: t=1.8741186035945496, p=0.06305944571167528
shape of training data (176, 243)
0.2
0.2
shape before pca: (176, 236)
number of pca components: 70
shape after pca: (176, 70)
0.2
shape before pca: (176, 236)
number of pca components: 70
shape after pca: (176, 70)
0.2
shape before pca: (176, 236)
number of pca components: 70
shape after pca: (176, 70)
0.2
shape of training data (176, 243)
TTest for residuals of model in lyon: t=1.0139092728950863, p=0.31411785324063163
shape of training data (861, 243)
0.01
0.01
shape before pca: (861, 236)
number of pca components: 134
shape after pca: (861, 134)
0.01
shape bef

,improvement_lasso,improvement_lasso_boosted,improvement_pca_lasso,improvement_pca_lasso_boosted,improvement_mean_model
marseille,41.680482,35.214125,44.438605,46.358819,46.929571
lyon,22.838334,41.320281,22.907843,40.834638,42.387262
paris,33.17937,36.796983,35.57732,29.094963,36.334713
hamburg,40.074417,50.268429,33.66026,41.923217,50.256995
berlin,30.558282,32.080985,30.441039,37.696938,38.398618
bremen,40.244172,54.756016,48.18371,46.799312,56.940528


In [10]:
results_r2#unemployment

,r2_naive,r2_lasso,r2_lasso_boosted,r2_pca_lasso,r2_pca_lasso_boosted,r2_mean_model,p_values
marseille,-0.001364,0.41601,0.351258,0.443628,0.462857,0.468572,0.063059
lyon,-0.001873,0.226938,0.412104,0.227634,0.407238,0.422793,0.314118
paris,-0.014116,0.322361,0.359048,0.346679,0.28094,0.35436,0.042977
hamburg,-0.124421,0.326184,0.440808,0.254062,0.346973,0.440679,0.100437
berlin,-0.00026,0.305402,0.320633,0.30423,0.376808,0.383826,0.345727
bremen,-0.043206,0.376624,0.528012,0.459449,0.445007,0.550801,0.074483


In [ ]:
results_mse

In [5]:
results_rmse = results_mse**(0.5)
results_rmse = results_rmse.rename(columns = {'mse_naive': 'rmse_naive','mse_lasso':'rmse_lasso', 'mse_lasso_boosted':'rmse_lasso_boosted', 'mse_pca_lasso':'rmse_pca_lasso', 'mse_pca_lasso_boosted':'rmse_pca_lasso_boosted', 'mse_mean_model':'rmse_mean_model'})
results_rmse

,rmse_naive,rmse_lasso,rmse_lasso_boosted,rmse_pca_lasso,rmse_pca_lasso_boosted,rmse_mean_model
marseille,0.104344,0.079684,0.083986,0.077777,0.076421,0.076014
lyon,0.051135,0.044918,0.039171,0.044898,0.039333,0.038813
paris,0.041161,0.033646,0.032723,0.033037,0.03466,0.032842
hamburg,0.0191,0.014786,0.01347,0.015557,0.014556,0.013471
berlin,0.029304,0.02442,0.02415,0.02444,0.02313,0.023
bremen,0.043799,0.033858,0.029461,0.031528,0.031947,0.028741


In [6]:
output = pd.concat([results, results_r2, results_mse, results_rmse], axis = 1, join = 'inner')
output = output[['mse_naive', 'rmse_naive', 'r2_naive',
               'mse_lasso', 'rmse_lasso', 'r2_lasso', 'improvement_lasso',
               'mse_lasso_boosted', 'rmse_lasso_boosted', 'r2_lasso_boosted', 'improvement_lasso_boosted',
               'mse_pca_lasso', 'rmse_pca_lasso', 'r2_pca_lasso', 'improvement_pca_lasso',
               'mse_pca_lasso_boosted', 'rmse_pca_lasso_boosted', 'r2_pca_lasso_boosted', 'improvement_pca_lasso_boosted',
               'mse_mean_model', 'rmse_mean_model', 'r2_mean_model', 'improvement_mean_model']]
output.to_excel(f'output/results/results_{target}_{dens_type}_{radius}.xlsx')
output

,mse_naive,rmse_naive,r2_naive,mse_lasso,rmse_lasso,r2_lasso,improvement_lasso,mse_lasso_boosted,rmse_lasso_boosted,r2_lasso_boosted,...,r2_pca_lasso,improvement_pca_lasso,mse_pca_lasso_boosted,rmse_pca_lasso_boosted,r2_pca_lasso_boosted,improvement_pca_lasso_boosted,mse_mean_model,rmse_mean_model,r2_mean_model,improvement_mean_model
marseille,0.010888,0.104344,-0.001364,0.00635,0.079684,0.41601,41.680482,0.007054,0.083986,0.351258,...,0.443628,44.438605,0.00584,0.076421,0.462857,46.358819,0.005778,0.076014,0.468572,46.929571
lyon,0.002615,0.051135,-0.001873,0.002018,0.044918,0.226938,22.838334,0.001534,0.039171,0.412104,...,0.227634,22.907843,0.001547,0.039333,0.407238,40.834638,0.001506,0.038813,0.422793,42.387262
paris,0.001694,0.041161,-0.014116,0.001132,0.033646,0.322361,33.17937,0.001071,0.032723,0.359048,...,0.346679,35.57732,0.001201,0.03466,0.28094,29.094963,0.001079,0.032842,0.35436,36.334713
hamburg,0.000365,0.0191,-0.124421,0.000219,0.014786,0.326184,40.074417,0.000181,0.01347,0.440808,...,0.254062,33.66026,0.000212,0.014556,0.346973,41.923217,0.000181,0.013471,0.440679,50.256995
berlin,0.000859,0.029304,-0.00026,0.000596,0.02442,0.305402,30.558282,0.000583,0.02415,0.320633,...,0.30423,30.441039,0.000535,0.02313,0.376808,37.696938,0.000529,0.023,0.383826,38.398618
bremen,0.001918,0.043799,-0.043206,0.001146,0.033858,0.376624,40.244172,0.000868,0.029461,0.528012,...,0.459449,48.18371,0.001021,0.031947,0.445007,46.799312,0.000826,0.028741,0.550801,56.940528


#### Foreign Nationals

In [12]:
results = pd.DataFrame(columns = ['improvement_lasso', 'improvement_lasso_boosted', 'improvement_pca_lasso', 'improvement_pca_lasso_boosted', 'improvement_mean_model'])
results_mse = pd.DataFrame(columns = ['mse_naive','mse_lasso', 'mse_lasso_boosted', 'mse_pca_lasso', 'mse_pca_lasso_boosted', 'mse_mean_model'])
results_r2 = pd.DataFrame(columns = ['r2_naive','r2_lasso', 'r2_lasso_boosted', 'r2_pca_lasso', 'r2_pca_lasso_boosted', 'r2_mean_model'])
p_list= []
target = 'foreign_nationals'
dens_type = 'count'
radius = 1000

for city in ['marseille', 'lyon', 'paris']:
    country = 'FR'
    agg = get_training_data(city, country, radius, dens_type,  2015)
    
    # Lasso:
    predicts = get_best_lasso_model(agg=agg, target=target, city=city, country=country, socio_year=2015, density_type=dens_type, radius = radius, output ='predicts')
    predicts_master = predicts.copy()
    predicts_master = predicts_master.rename(columns = {'y_pred': 'y_pred_lasso'})

    # Lasso Boosted
    cols = get_best_lasso_model(agg=agg, target=target, city=city, country=country, socio_year=2015, density_type=dens_type, radius = radius, output ='used_columns')
    predicts = train_xgboost(agg, target, cols, 'predicts')
    predicts_master.loc[:, 'y_pred_lasso_boosted'] = predicts.y_pred

    # PCA Lasso
    predicts = get_best_pca_lasso_model(agg=agg, target=target, city=city, country=country, socio_year=2015, density_type=dens_type, radius = radius,output = 'predicts')
    predicts_master.loc[:,'y_pred_pca_lasso'] = predicts.y_pred

    # PCA Lasso boosted
    comps = get_best_pca_lasso_model(agg=agg,target=target, city=city, country=country, socio_year=2015,  density_type=dens_type, radius = radius, output = 'components')
    cols = get_best_pca_lasso_model(agg=agg,target=target, city=city, country=country, socio_year=2015,  density_type=dens_type, radius = radius, output = 'used_columns')
    reduced_data = pd.DataFrame(comps)
    reduced_data = reduced_data.join(agg.iloc[:,-5:])
    predicts = train_xgboost(reduced_data, target, cols, 'predicts')
    predicts_master.loc[:,'y_pred_pca_lasso_boosted'] = predicts.y_pred

    #predicts_master.loc[:, 'y_pred_mean_model'] = predicts_master[['y_pred_lasso', 'y_pred_lasso_boosted', 'y_pred_pca_lasso', 'y_pred_pca_lasso_boosted']].mean(axis = 1)
    predicts_master.loc[:, 'y_pred_mean_model'] = predicts_master[['y_pred_lasso_boosted', 'y_pred_pca_lasso_boosted']].mean(axis = 1)


    targets = ['unemployment_rate', 'income_levels', 'foreign_nationals']
    scaler = get_training_data(city, country, radius, dens_type, 2015, 'scaler')
    scaler_new = RobustScaler()
    scaler_new.center_, scaler_new.scale_ = scaler.center_[targets.index(target)], scaler.scale_[targets.index(target)]
    predicts_master = pd.DataFrame(scaler_new.inverse_transform(predicts_master), columns =predicts_master.columns)


    naive_mse = metrics.mean_squared_error(predicts_master.y_test, predicts_master.naive) 
    mse_lasso = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_lasso)
    mse_lasso_boosted = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_lasso_boosted)
    mse_pca_lasso = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_pca_lasso)
    mse_pca_lasso_boosted = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_pca_lasso_boosted)
    mse_mean_model = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_mean_model)
    
    naive_r2 = metrics.r2_score(predicts_master.y_test, predicts_master.naive) 
    r2_lasso = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_lasso)
    r2_lasso_boosted = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_lasso_boosted)
    r2_pca_lasso = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_pca_lasso)
    r2_pca_lasso_boosted = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_pca_lasso_boosted)
    r2_mean_model = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_mean_model)

    improvement_lasso = 100 - (mse_lasso/naive_mse)*100
    improvement_lasso_boosted = 100 - (mse_lasso_boosted /naive_mse)*100
    improvement_pca_lasso = 100 - (mse_pca_lasso/naive_mse)*100
    improvement_pca_lasso_boosted = 100 - (mse_pca_lasso_boosted/naive_mse)*100
    improvement_mean_model = 100 - (mse_mean_model/naive_mse)*100
    results.loc[city,:]=[improvement_lasso, improvement_lasso_boosted, improvement_pca_lasso, improvement_pca_lasso_boosted, improvement_mean_model]
    results_mse.loc[city,:]=[naive_mse, mse_lasso, mse_lasso_boosted, mse_pca_lasso, mse_pca_lasso_boosted, mse_mean_model]
    results_r2.loc[city,:]=[naive_r2, r2_lasso, r2_lasso_boosted, r2_pca_lasso, r2_pca_lasso_boosted, r2_mean_model]
    
    residuals = pd.DataFrame(columns = ['residuals_naive', 'residuals_m5'])
    residuals.loc[:,'residuals_naive'] = (predicts_master.y_test-predicts_master.naive)**2
    residuals.loc[:,'residuals_m5'] = (predicts_master.y_test-predicts_master.y_pred_mean_model)**2

    t_stat, p = stats.ttest_ind(residuals['residuals_naive'], residuals['residuals_m5'])
    print(f'TTest for residuals of model in {city}: t={t_stat}, p={p}')
    p_list.append(p)

    
for city in ['hamburg', 'berlin', 'bremen']:
    country = 'DE'
    agg = get_training_data(city, country, radius, dens_type,  2015)
    
    # Lasso:
    predicts = get_best_lasso_model(agg=agg, target=target, city=city, country=country, socio_year=2015, density_type=dens_type, radius = radius, output ='predicts')
    predicts_master = predicts.copy()
    predicts_master = predicts_master.rename(columns = {'y_pred': 'y_pred_lasso'})

    # Lasso Boosted
    cols = get_best_lasso_model(agg=agg, target=target, city=city, country=country, socio_year=2015, density_type=dens_type, radius = radius, output ='used_columns')
    predicts = train_xgboost(agg, target, cols, 'predicts')
    predicts_master.loc[:, 'y_pred_lasso_boosted'] = predicts.y_pred

    # PCA Lasso
    predicts = get_best_pca_lasso_model(agg=agg, target=target, city=city, country=country, socio_year=2015,  density_type=dens_type, radius = radius,output = 'predicts')
    predicts_master.loc[:,'y_pred_pca_lasso'] = predicts.y_pred

    # PCA Lasso boosted
    comps = get_best_pca_lasso_model(agg=agg,target=target, city=city, country=country, socio_year=2015,  density_type=dens_type, radius = radius, output = 'components')
    cols = get_best_pca_lasso_model(agg=agg,target=target, city=city, country=country, socio_year=2015, density_type=dens_type, radius = radius, output = 'used_columns')
    reduced_data = pd.DataFrame(comps)
    reduced_data = reduced_data.join(agg.iloc[:,-5:])
    predicts = train_xgboost(reduced_data, target, cols, 'predicts')
    predicts_master.loc[:,'y_pred_pca_lasso_boosted'] = predicts.y_pred

    #predicts_master.loc[:, 'y_pred_mean_model'] = predicts_master[['y_pred_lasso', 'y_pred_lasso_boosted', 'y_pred_pca_lasso', 'y_pred_pca_lasso_boosted']].mean(axis = 1)
    predicts_master.loc[:, 'y_pred_mean_model'] = predicts_master[['y_pred_lasso_boosted', 'y_pred_pca_lasso_boosted']].mean(axis = 1)

    targets = ['unemployment_rate', 'income_levels', 'foreign_nationals']
    scaler = get_training_data(city, country, radius, dens_type, 2015, 'scaler')
    scaler_new = RobustScaler()
    scaler_new.center_, scaler_new.scale_ = scaler.center_[targets.index(target)], scaler.scale_[targets.index(target)]
    predicts_master = pd.DataFrame(scaler_new.inverse_transform(predicts_master), columns =predicts_master.columns)


    naive_mse = metrics.mean_squared_error(predicts_master.y_test, predicts_master.naive) 
    mse_lasso = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_lasso)
    mse_lasso_boosted = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_lasso_boosted)
    mse_pca_lasso = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_pca_lasso)
    mse_pca_lasso_boosted = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_pca_lasso_boosted)
    mse_mean_model = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_mean_model)
    
    naive_r2 = metrics.r2_score(predicts_master.y_test, predicts_master.naive) 
    r2_lasso = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_lasso)
    r2_lasso_boosted = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_lasso_boosted)
    r2_pca_lasso = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_pca_lasso)
    r2_pca_lasso_boosted = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_pca_lasso_boosted)
    r2_mean_model = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_mean_model)

    improvement_lasso = 100 - (mse_lasso/naive_mse)*100
    improvement_lasso_boosted = 100 - (mse_lasso_boosted /naive_mse)*100
    improvement_pca_lasso = 100 - (mse_pca_lasso/naive_mse)*100
    improvement_pca_lasso_boosted = 100 - (mse_pca_lasso_boosted/naive_mse)*100
    improvement_mean_model = 100 - (mse_mean_model/naive_mse)*100
    results.loc[city,:]=[improvement_lasso, improvement_lasso_boosted, improvement_pca_lasso, improvement_pca_lasso_boosted, improvement_mean_model]
    results_mse.loc[city,:]=[naive_mse, mse_lasso, mse_lasso_boosted, mse_pca_lasso, mse_pca_lasso_boosted, mse_mean_model]
    results_r2.loc[city,:]=[naive_r2, r2_lasso, r2_lasso_boosted, r2_pca_lasso, r2_pca_lasso_boosted, r2_mean_model]
    
    residuals = pd.DataFrame(columns = ['residuals_naive', 'residuals_m5'])
    residuals.loc[:,'residuals_naive'] = (predicts_master.y_test-predicts_master.naive)**2
    residuals.loc[:,'residuals_m5'] = (predicts_master.y_test-predicts_master.y_pred_mean_model)**2

    t_stat, p = stats.ttest_ind(residuals['residuals_naive'], residuals['residuals_m5'])
    print(f'TTest for residuals of model in {city}: t={t_stat}, p={p}')
    p_list.append(p)

results_r2.loc[:,'p_values'] = p_list
    
#results.loc['mean'] = results.mean()
results

shape of training data (344, 243)
0.01
0.01
shape before pca: (344, 236)
number of pca components: 69
shape after pca: (344, 69)
0.01
shape before pca: (344, 236)
number of pca components: 69
shape after pca: (344, 69)
0.01
shape before pca: (344, 236)
number of pca components: 69
shape after pca: (344, 69)
0.01
shape of training data (344, 243)
TTest for residuals of model in marseille: t=2.787513355558777, p=0.006072149367044044
shape of training data (176, 243)
0.1
0.1
shape before pca: (176, 236)
number of pca components: 70
shape after pca: (176, 70)
0.2
shape before pca: (176, 236)
number of pca components: 70
shape after pca: (176, 70)
0.2
shape before pca: (176, 236)
number of pca components: 70
shape after pca: (176, 70)
0.2
shape of training data (176, 243)
TTest for residuals of model in lyon: t=0.5866145234907076, p=0.5593497160186528
shape of training data (861, 243)
0.01
0.01
shape before pca: (861, 236)
number of pca components: 134
shape after pca: (861, 134)
0.01
shape

,improvement_lasso,improvement_lasso_boosted,improvement_pca_lasso,improvement_pca_lasso_boosted,improvement_mean_model
marseille,70.37185,75.275179,72.938022,63.059867,72.405118
lyon,26.652563,25.031563,26.075534,26.925843,27.669349
paris,30.349951,34.698283,30.145525,30.013002,35.326112
hamburg,41.669386,11.528567,24.711452,-4.445448,25.432159
berlin,62.771734,59.713055,56.644192,64.859589,67.603515
bremen,19.852717,29.928093,19.422933,28.074484,32.320905


In [13]:
results_r2#foreign nationals

,r2_naive,r2_lasso,r2_lasso_boosted,r2_pca_lasso,r2_pca_lasso_boosted,r2_mean_model,p_values
marseille,-0.026125,0.695978,0.746292,0.72231,0.620948,0.716842,0.006072
lyon,-0.002119,0.264972,0.248727,0.259189,0.26771,0.275161,0.559350
paris,-0.021716,0.288374,0.332802,0.286286,0.284932,0.339217,0.098296
hamburg,-0.26129,0.264282,-0.115881,0.050393,-0.31736,0.059483,0.370972
berlin,-0.033907,0.615094,0.58347,0.551741,0.636681,0.66505,0.039152
bremen,-0.012699,0.18835,0.290383,0.183997,0.271611,0.314615,0.392515


In [10]:
results_mse

,mse_naive,mse_lasso,mse_lasso_boosted,mse_pca_lasso,mse_pca_lasso_boosted,mse_mean_model
marseille,0.009139,0.002708,0.002259,0.002473,0.003376,0.002522
lyon,0.002496,0.001831,0.001871,0.001845,0.001824,0.001806
paris,0.002873,0.002001,0.001876,0.002007,0.002011,0.001858
hamburg,0.005245,0.003059,0.00464,0.003949,0.005478,0.003911
berlin,0.003961,0.001475,0.001596,0.001717,0.001392,0.001283
bremen,0.006663,0.00534,0.004669,0.005369,0.004793,0.00451


In [11]:
results_rmse = results_mse**(0.5)
results_rmse = results_rmse.rename(columns = {'mse_naive': 'rmse_naive','mse_lasso':'rmse_lasso', 'mse_lasso_boosted':'rmse_lasso_boosted', 'mse_pca_lasso':'rmse_pca_lasso', 'mse_pca_lasso_boosted':'rmse_pca_lasso_boosted', 'mse_mean_model':'rmse_mean_model'})
results_rmse

,rmse_naive,rmse_lasso,rmse_lasso_boosted,rmse_pca_lasso,rmse_pca_lasso_boosted,rmse_mean_model
marseille,0.095596,0.052034,0.047534,0.04973,0.058102,0.050217
lyon,0.049962,0.042789,0.043259,0.042957,0.042709,0.042492
paris,0.053601,0.044734,0.043315,0.044799,0.044842,0.043106
hamburg,0.072423,0.055312,0.06812,0.06284,0.074015,0.062539
berlin,0.062939,0.038402,0.039949,0.041443,0.03731,0.035824
bremen,0.081628,0.073078,0.06833,0.073274,0.069228,0.067153


In [12]:
output = pd.concat([results, results_r2, results_mse, results_rmse], axis = 1, join = 'inner')
output = output[['mse_naive', 'rmse_naive', 'r2_naive',
               'mse_lasso', 'rmse_lasso', 'r2_lasso', 'improvement_lasso',
               'mse_lasso_boosted', 'rmse_lasso_boosted', 'r2_lasso_boosted', 'improvement_lasso_boosted',
               'mse_pca_lasso', 'rmse_pca_lasso', 'r2_pca_lasso', 'improvement_pca_lasso',
               'mse_pca_lasso_boosted', 'rmse_pca_lasso_boosted', 'r2_pca_lasso_boosted', 'improvement_pca_lasso_boosted',
               'mse_mean_model', 'rmse_mean_model', 'r2_mean_model', 'improvement_mean_model']]
output.to_excel(f'output/results/results_{target}_{dens_type}_{radius}.xlsx')
output

,mse_naive,rmse_naive,r2_naive,mse_lasso,rmse_lasso,r2_lasso,improvement_lasso,mse_lasso_boosted,rmse_lasso_boosted,r2_lasso_boosted,...,r2_pca_lasso,improvement_pca_lasso,mse_pca_lasso_boosted,rmse_pca_lasso_boosted,r2_pca_lasso_boosted,improvement_pca_lasso_boosted,mse_mean_model,rmse_mean_model,r2_mean_model,improvement_mean_model
marseille,0.009139,0.095596,-0.026125,0.002708,0.052034,0.695978,70.37185,0.002259,0.047534,0.746292,...,0.72231,72.938022,0.003376,0.058102,0.620948,63.059867,0.002522,0.050217,0.716842,72.405118
lyon,0.002496,0.049962,-0.002119,0.001831,0.042789,0.264972,26.652563,0.001871,0.043259,0.248727,...,0.259189,26.075534,0.001824,0.042709,0.26771,26.925843,0.001806,0.042492,0.275161,27.669349
paris,0.002873,0.053601,-0.021716,0.002001,0.044734,0.288374,30.349951,0.001876,0.043315,0.332802,...,0.286286,30.145525,0.002011,0.044842,0.284932,30.013002,0.001858,0.043106,0.339217,35.326112
hamburg,0.005245,0.072423,-0.26129,0.003059,0.055312,0.264282,41.669386,0.00464,0.06812,-0.115881,...,0.050393,24.711452,0.005478,0.074015,-0.31736,-4.445448,0.003911,0.062539,0.059483,25.432159
berlin,0.003961,0.062939,-0.033907,0.001475,0.038402,0.615094,62.771734,0.001596,0.039949,0.58347,...,0.551741,56.644192,0.001392,0.03731,0.636681,64.859589,0.001283,0.035824,0.66505,67.603515
bremen,0.006663,0.081628,-0.012699,0.00534,0.073078,0.18835,19.852717,0.004669,0.06833,0.290383,...,0.183997,19.422933,0.004793,0.069228,0.271611,28.074484,0.00451,0.067153,0.314615,32.320905


#### Income levels

In [15]:
results = pd.DataFrame(columns = ['improvement_lasso', 'improvement_lasso_boosted', 'improvement_pca_lasso', 'improvement_pca_lasso_boosted', 'improvement_mean_model'])
results_mse = pd.DataFrame(columns = ['mse_naive','mse_lasso', 'mse_lasso_boosted', 'mse_pca_lasso', 'mse_pca_lasso_boosted', 'mse_mean_model'])
results_r2 = pd.DataFrame(columns = ['r2_naive','r2_lasso', 'r2_lasso_boosted', 'r2_pca_lasso', 'r2_pca_lasso_boosted', 'r2_mean_model'])
p_list = []
target = 'income_levels'
dens_type = 'count'
radius = 1000

for city in ['marseille', 'lyon', 'paris']:
    country = 'FR'
    agg = get_training_data(city, country, radius, dens_type,  2015)
    
    # Lasso:
    predicts = get_best_lasso_model(agg=agg, target=target, city=city, country=country, socio_year=2015, density_type=dens_type, radius = radius, output ='predicts')
    predicts_master = predicts.copy()
    predicts_master = predicts_master.rename(columns = {'y_pred': 'y_pred_lasso'})

    # Lasso Boosted
    cols = get_best_lasso_model(agg=agg, target=target, city=city, country=country, socio_year=2015, density_type=dens_type, radius = radius, output ='used_columns')
    predicts = train_xgboost(agg, target, cols, 'predicts')
    predicts_master.loc[:, 'y_pred_lasso_boosted'] = predicts.y_pred

    # PCA Lasso
    predicts = get_best_pca_lasso_model(agg=agg, target=target, city=city, country=country, socio_year=2015, density_type=dens_type, radius = radius,output = 'predicts')
    predicts_master.loc[:,'y_pred_pca_lasso'] = predicts.y_pred

    # PCA Lasso boosted
    comps = get_best_pca_lasso_model(agg=agg,target=target, city=city, country=country, socio_year=2015,  density_type=dens_type, radius = radius, output = 'components')
    cols = get_best_pca_lasso_model(agg=agg,target=target, city=city, country=country, socio_year=2015,  density_type=dens_type, radius = radius, output = 'used_columns')
    reduced_data = pd.DataFrame(comps)
    reduced_data = reduced_data.join(agg.iloc[:,-5:])
    predicts = train_xgboost(reduced_data, target, cols, 'predicts')
    predicts_master.loc[:,'y_pred_pca_lasso_boosted'] = predicts.y_pred

    #predicts_master.loc[:, 'y_pred_mean_model'] = predicts_master[['y_pred_lasso', 'y_pred_lasso_boosted', 'y_pred_pca_lasso', 'y_pred_pca_lasso_boosted']].mean(axis = 1)
    predicts_master.loc[:, 'y_pred_mean_model'] = predicts_master[['y_pred_lasso_boosted', 'y_pred_pca_lasso_boosted']].mean(axis = 1)


    targets = ['unemployment_rate', 'income_levels', 'foreign_nationals']
    scaler = get_training_data(city, country, radius, dens_type, 2015, 'scaler')
    scaler_new = RobustScaler()
    scaler_new.center_, scaler_new.scale_ = scaler.center_[targets.index(target)], scaler.scale_[targets.index(target)]
    predicts_master = pd.DataFrame(scaler_new.inverse_transform(predicts_master), columns =predicts_master.columns)


    naive_mse = metrics.mean_squared_error(predicts_master.y_test, predicts_master.naive) 
    mse_lasso = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_lasso)
    mse_lasso_boosted = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_lasso_boosted)
    mse_pca_lasso = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_pca_lasso)
    mse_pca_lasso_boosted = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_pca_lasso_boosted)
    mse_mean_model = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_mean_model)
    
    naive_r2 = metrics.r2_score(predicts_master.y_test, predicts_master.naive) 
    r2_lasso = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_lasso)
    r2_lasso_boosted = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_lasso_boosted)
    r2_pca_lasso = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_pca_lasso)
    r2_pca_lasso_boosted = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_pca_lasso_boosted)
    r2_mean_model = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_mean_model)

    improvement_lasso = 100 - (mse_lasso/naive_mse)*100
    improvement_lasso_boosted = 100 - (mse_lasso_boosted /naive_mse)*100
    improvement_pca_lasso = 100 - (mse_pca_lasso/naive_mse)*100
    improvement_pca_lasso_boosted = 100 - (mse_pca_lasso_boosted/naive_mse)*100
    improvement_mean_model = 100 - (mse_mean_model/naive_mse)*100
    results.loc[city,:]=[improvement_lasso, improvement_lasso_boosted, improvement_pca_lasso, improvement_pca_lasso_boosted, improvement_mean_model]
    results_mse.loc[city,:]=[naive_mse, mse_lasso, mse_lasso_boosted, mse_pca_lasso, mse_pca_lasso_boosted, mse_mean_model]
    results_r2.loc[city,:]=[naive_r2, r2_lasso, r2_lasso_boosted, r2_pca_lasso, r2_pca_lasso_boosted, r2_mean_model]
    
    residuals = pd.DataFrame(columns = ['residuals_naive', 'residuals_m5'])
    residuals.loc[:,'residuals_naive'] = (predicts_master.y_test-predicts_master.naive)**2
    residuals.loc[:,'residuals_m5'] = (predicts_master.y_test-predicts_master.y_pred_mean_model)**2

    t_stat, p = stats.ttest_ind(residuals['residuals_naive'], residuals['residuals_m5'])
    print(f'TTest for residuals of model in {city}: t={t_stat}, p={p}')
    p_list.append(p)

    
for city in ['hamburg', 'berlin', 'bremen']:
    country = 'DE'
    agg = get_training_data(city, country, radius, dens_type,  2015)
    
    # Lasso:
    predicts = get_best_lasso_model(agg=agg, target=target, city=city, country=country, socio_year=2015, density_type=dens_type, radius = radius, output ='predicts')
    predicts_master = predicts.copy()
    predicts_master = predicts_master.rename(columns = {'y_pred': 'y_pred_lasso'})

    # Lasso Boosted
    cols = get_best_lasso_model(agg=agg, target=target, city=city, country=country, socio_year=2015, density_type=dens_type, radius = radius, output ='used_columns')
    predicts = train_xgboost(agg, target, cols, 'predicts')
    predicts_master.loc[:, 'y_pred_lasso_boosted'] = predicts.y_pred

    # PCA Lasso
    predicts = get_best_pca_lasso_model(agg=agg, target=target, city=city, country=country, socio_year=2015,  density_type=dens_type, radius = radius,output = 'predicts')
    predicts_master.loc[:,'y_pred_pca_lasso'] = predicts.y_pred

    # PCA Lasso boosted
    comps = get_best_pca_lasso_model(agg=agg,target=target, city=city, country=country, socio_year=2015,  density_type=dens_type, radius = radius, output = 'components')
    cols = get_best_pca_lasso_model(agg=agg,target=target, city=city, country=country, socio_year=2015, density_type=dens_type, radius = radius, output = 'used_columns')
    reduced_data = pd.DataFrame(comps)
    reduced_data = reduced_data.join(agg.iloc[:,-5:])
    predicts = train_xgboost(reduced_data, target, cols, 'predicts')
    predicts_master.loc[:,'y_pred_pca_lasso_boosted'] = predicts.y_pred

    #predicts_master.loc[:, 'y_pred_mean_model'] = predicts_master[['y_pred_lasso', 'y_pred_lasso_boosted', 'y_pred_pca_lasso', 'y_pred_pca_lasso_boosted']].mean(axis = 1)
    predicts_master.loc[:, 'y_pred_mean_model'] = predicts_master[['y_pred_lasso_boosted', 'y_pred_pca_lasso_boosted']].mean(axis = 1)

    targets = ['unemployment_rate', 'income_levels', 'foreign_nationals']
    scaler = get_training_data(city, country, radius, dens_type, 2015, 'scaler')
    scaler_new = RobustScaler()
    scaler_new.center_, scaler_new.scale_ = scaler.center_[targets.index(target)], scaler.scale_[targets.index(target)]
    predicts_master = pd.DataFrame(scaler_new.inverse_transform(predicts_master), columns =predicts_master.columns)


    naive_mse = metrics.mean_squared_error(predicts_master.y_test, predicts_master.naive) 
    mse_lasso = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_lasso)
    mse_lasso_boosted = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_lasso_boosted)
    mse_pca_lasso = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_pca_lasso)
    mse_pca_lasso_boosted = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_pca_lasso_boosted)
    mse_mean_model = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_mean_model)
    
    naive_r2 = metrics.r2_score(predicts_master.y_test, predicts_master.naive) 
    r2_lasso = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_lasso)
    r2_lasso_boosted = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_lasso_boosted)
    r2_pca_lasso = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_pca_lasso)
    r2_pca_lasso_boosted = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_pca_lasso_boosted)
    r2_mean_model = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_mean_model)

    improvement_lasso = 100 - (mse_lasso/naive_mse)*100
    improvement_lasso_boosted = 100 - (mse_lasso_boosted /naive_mse)*100
    improvement_pca_lasso = 100 - (mse_pca_lasso/naive_mse)*100
    improvement_pca_lasso_boosted = 100 - (mse_pca_lasso_boosted/naive_mse)*100
    improvement_mean_model = 100 - (mse_mean_model/naive_mse)*100
    results.loc[city,:]=[improvement_lasso, improvement_lasso_boosted, improvement_pca_lasso, improvement_pca_lasso_boosted, improvement_mean_model]
    results_mse.loc[city,:]=[naive_mse, mse_lasso, mse_lasso_boosted, mse_pca_lasso, mse_pca_lasso_boosted, mse_mean_model]
    results_r2.loc[city,:]=[naive_r2, r2_lasso, r2_lasso_boosted, r2_pca_lasso, r2_pca_lasso_boosted, r2_mean_model]
    
    residuals = pd.DataFrame(columns = ['residuals_naive', 'residuals_m5'])
    residuals.loc[:,'residuals_naive'] = (predicts_master.y_test-predicts_master.naive)**2
    residuals.loc[:,'residuals_m5'] = (predicts_master.y_test-predicts_master.y_pred_mean_model)**2

    t_stat, p = stats.ttest_ind(residuals['residuals_naive'], residuals['residuals_m5'])
    print(f'TTest for residuals of model in {city}: t={t_stat}, p={p}')
    p_list.append(p)

results_r2.loc[:,'p_values'] = p_list    
#results.loc['mean'] = results.mean()
results

shape of training data (344, 243)
0.01
0.01
shape before pca: (344, 236)
number of pca components: 69
shape after pca: (344, 69)
0.01
shape before pca: (344, 236)
number of pca components: 69
shape after pca: (344, 69)
0.01
shape before pca: (344, 236)
number of pca components: 69
shape after pca: (344, 69)
0.01
shape of training data (344, 243)
TTest for residuals of model in marseille: t=5.152169529576394, p=8.860793693545753e-07
shape of training data (176, 243)
0.1
0.1
shape before pca: (176, 236)
number of pca components: 70
shape after pca: (176, 70)
0.1
shape before pca: (176, 236)
number of pca components: 70
shape after pca: (176, 70)
0.1
shape before pca: (176, 236)
number of pca components: 70
shape after pca: (176, 70)
0.1
shape of training data (176, 243)
TTest for residuals of model in lyon: t=2.0535956840403324, p=0.04375225090153381
shape of training data (861, 243)
0.01
0.01
shape before pca: (861, 236)
number of pca components: 134
shape after pca: (861, 134)
0.01
sha

,improvement_lasso,improvement_lasso_boosted,improvement_pca_lasso,improvement_pca_lasso_boosted,improvement_mean_model
marseille,74.639366,74.491927,76.171372,67.429672,75.309903
lyon,42.887151,52.348249,48.834856,53.140459,57.652806
paris,71.30661,75.369685,71.739564,70.404145,74.625626
hamburg,38.956691,37.21714,36.641392,39.778275,47.218826
berlin,17.489254,39.874772,-27.711928,-22.642964,29.731272
bremen,32.959338,40.586651,31.282117,40.09886,45.940758


In [16]:
results_r2#income levels

,r2_naive,r2_lasso,r2_lasso_boosted,r2_pca_lasso,r2_pca_lasso_boosted,r2_mean_model,p_values
marseille,-0.000792,0.746193,0.744717,0.761525,0.674039,0.752903,8.860794e-07
lyon,-0.000858,0.428381,0.523073,0.487909,0.531002,0.576165,4.375225e-02
paris,-0.005564,0.71147,0.752326,0.715823,0.702395,0.744844,1.924534e-11
hamburg,-0.042585,0.363572,0.345436,0.339433,0.372138,0.449712,1.974557e-01
berlin,-0.001062,0.174016,0.398109,-0.278476,-0.227733,0.296566,3.379441e-01
bremen,-0.205356,0.191921,0.283858,0.171705,0.277978,0.348394,1.031173e-01


In [15]:
results_mse

,mse_naive,mse_lasso,mse_lasso_boosted,mse_pca_lasso,mse_pca_lasso_boosted,mse_mean_model
marseille,361248.317935,91614.863765,92147.484676,86080.519236,117659.760747,89192.560479
lyon,244401.878031,139584.874445,116461.773642,125048.573088,114525.59882,103497.3381
paris,840140.3386,241064.743189,206929.208861,237427.324887,248646.716541,213180.354353
hamburg,667237.806043,407304.032396,418910.978417,422752.587952,401822.113891,352175.950401
berlin,731046.943231,603192.289891,439543.640732,933634.145608,896577.641375,513697.385316
bremen,626827.178684,420229.092606,372419.020751,430742.366263,375476.626216,338858.019679


In [16]:
results_rmse = results_mse**(0.5)
results_rmse = results_rmse.rename(columns = {'mse_naive': 'rmse_naive','mse_lasso':'rmse_lasso', 'mse_lasso_boosted':'rmse_lasso_boosted', 'mse_pca_lasso':'rmse_pca_lasso', 'mse_pca_lasso_boosted':'rmse_pca_lasso_boosted', 'mse_mean_model':'rmse_mean_model'})
results_rmse

,rmse_naive,rmse_lasso,rmse_lasso_boosted,rmse_pca_lasso,rmse_pca_lasso_boosted,rmse_mean_model
marseille,601.039365,302.679474,303.558042,293.394818,343.015686,298.651236
lyon,494.370183,373.610592,341.264961,353.622077,338.41631,321.710022
paris,916.591697,490.983445,454.894723,487.265148,498.64488,461.714581
hamburg,816.846256,638.203755,647.233326,650.194269,633.894403,593.444143
berlin,855.012832,776.65455,662.980875,966.247456,946.877839,716.726855
bremen,791.724181,648.250795,610.261436,656.309657,612.761476,582.115126


In [17]:
output = pd.concat([results, results_r2, results_mse, results_rmse], axis = 1, join = 'inner')
output = output[['mse_naive', 'rmse_naive', 'r2_naive',
               'mse_lasso', 'rmse_lasso', 'r2_lasso', 'improvement_lasso',
               'mse_lasso_boosted', 'rmse_lasso_boosted', 'r2_lasso_boosted', 'improvement_lasso_boosted',
               'mse_pca_lasso', 'rmse_pca_lasso', 'r2_pca_lasso', 'improvement_pca_lasso',
               'mse_pca_lasso_boosted', 'rmse_pca_lasso_boosted', 'r2_pca_lasso_boosted', 'improvement_pca_lasso_boosted',
               'mse_mean_model', 'rmse_mean_model', 'r2_mean_model', 'improvement_mean_model']]
output.to_excel(f'output/results/results_{target}_{dens_type}_{radius}.xlsx')
output

,mse_naive,rmse_naive,r2_naive,mse_lasso,rmse_lasso,r2_lasso,improvement_lasso,mse_lasso_boosted,rmse_lasso_boosted,r2_lasso_boosted,...,r2_pca_lasso,improvement_pca_lasso,mse_pca_lasso_boosted,rmse_pca_lasso_boosted,r2_pca_lasso_boosted,improvement_pca_lasso_boosted,mse_mean_model,rmse_mean_model,r2_mean_model,improvement_mean_model
marseille,361248.317935,601.039365,-0.000792,91614.863765,302.679474,0.746193,74.639366,92147.484676,303.558042,0.744717,...,0.761525,76.171372,117659.760747,343.015686,0.674039,67.429672,89192.560479,298.651236,0.752903,75.309903
lyon,244401.878031,494.370183,-0.000858,139584.874445,373.610592,0.428381,42.887151,116461.773642,341.264961,0.523073,...,0.487909,48.834856,114525.59882,338.41631,0.531002,53.140459,103497.3381,321.710022,0.576165,57.652806
paris,840140.3386,916.591697,-0.005564,241064.743189,490.983445,0.71147,71.30661,206929.208861,454.894723,0.752326,...,0.715823,71.739564,248646.716541,498.64488,0.702395,70.404145,213180.354353,461.714581,0.744844,74.625626
hamburg,667237.806043,816.846256,-0.042585,407304.032396,638.203755,0.363572,38.956691,418910.978417,647.233326,0.345436,...,0.339433,36.641392,401822.113891,633.894403,0.372138,39.778275,352175.950401,593.444143,0.449712,47.218826
berlin,731046.943231,855.012832,-0.001062,603192.289891,776.65455,0.174016,17.489254,439543.640732,662.980875,0.398109,...,-0.278476,-27.711928,896577.641375,946.877839,-0.227733,-22.642964,513697.385316,716.726855,0.296566,29.731272
bremen,626827.178684,791.724181,-0.205356,420229.092606,648.250795,0.191921,32.959338,372419.020751,610.261436,0.283858,...,0.171705,31.282117,375476.626216,612.761476,0.277978,40.09886,338858.019679,582.115126,0.348394,45.940758
